Laboration för kursen DD1334
Använder SQL-lite för att kommunicera med en databas som sparar data från vårt projekt i kartgenerering

Först måste vi hämta hem map klasser från github och installera dependencies

In [ ]:
! pip install depend #fixa sen
%pip install ipywidgets
! git clone git@github.com:poskusen/dd1334.git

Vi börjar med att generera en karta som ska sparas.

In [4]:
from Classes.map import Map
import ipywidgets as widgets
from IPython.display import display, clear_output
from draw_PNG import draw_map
ny_karta = Map(5, 50, 0, 0 ,100, realm_name= 'test_karta') # Skapa en karta med bara byar

draw_map(ny_karta)
output = widgets.Output()
with output: # Visa upp kartan
    image = open("test_image.png", "rb").read() 
    
    image_widget = widgets.Image(
        value=image,
        format='png',
        width=1000,            
        height=1000
    )
    display(image_widget)
display(output)


Output()

Sen connectar vi till databasen och skapar ett nytt table för kartor, continenter och allt content på kontinenterna.

Här uppstår ett stort problem som vi undviker genom att använda pickle.
För att spara vår representation av kontinenten så måste vi spara en lista med punkter. Detta blir väldigt tricky då SQL databaser är baserade på att man sparar 1 värde och inte en lista med värden. Något vi undviker när vi bara serialiserar hela objektet. 
En lösning hade varit skapa ett table per lista och därmed en per kontinent, men det är väldigt bökigt och förstör hela poängen med databasen. Och eftersom vi aldrig vill ha enbart en eller flera punkter från kontinent-listan finns det ingen poäng med att skapa ett helt nytt table, då vi aldrig hämtar enskilda värden från listan. Därför kan man se listan snarare som ett objekt, än en lista. Därför serialiserar vi listan och sparar den sen, allt annat hade varit galenskap.

För att ändå visa att det går gör vi det för städerna.
Vi sparar även bara städerna för floder och berg hade inneburit dubbelarbete, och visar egentligen ingenting.

Den enda fördelen med att köra sql är att det går att hämta kontinenter med rätt 'size' från databasen, dvs att jämföra storleken på serilizationen av points_list.

In [34]:
import sqlite3, csv

conn = sqlite3.connect('database.db')
cursor = conn.cursor()
cursor.execute('PRAGMA foreign_keys = ON;')
try:
    cursor.execute('DROP TABLE maps')
    cursor.execute('DROP TABLE continents')
    cursor.execute('DROP TABLE villages')
except:
    pass

cursor.execute('''CREATE TABLE maps(
               id INTEGER PRIMARY KEY AUTOINCREMENT,
               map_size_x INT DEFAULT 1000,
               map_size_y INT DEFAULT 1000,
               name VARCHAR(255) NOT NULL,
               amount_continents INTEGER NOT NULL
               );''') # Skapa ett table för kartor

cursor.execute('''CREATE TABLE continents(
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                points_serialised VARCHAR(65535),
                id_map INTEGER NOT NULL,
                FOREIGN KEY (id_map) REFERENCES maps(id)
               );''') # Skapa ett table för kontintenter


cursor.execute('''CREATE TABLE villages(
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                x INT NOT NULL,
                y INT NOT NULL,
                id_cont INTEGER NOT NULL,
                FOREIGN KEY (id_cont) REFERENCES continents(id)
               );''') # Skapa ett table för byar



OperationalError: table maps already exists

Stäng databas om det behövs

In [11]:
conn.close()

Skapa en enkel serialiserings algoritm som kan spara listor och en som kan återskapa innehållet och göra det till en lista

In [5]:
def stringify(continent_list):
    ser = ''
    for point in continent_list:
        str_point = str(point)
        str_point = str_point.strip('()')
        str_point = str_point.replace(' ','')
        ser +=  str_point + ','
    return ser

    

def unstringify(ser_cont):
    continent_list = []
    lista_nummer = ser_cont.split(',')
    print(lista_nummer)
    for i in range(0, len(lista_nummer) - 1, 2):
        continent_list.append((int(lista_nummer[i]), int(lista_nummer[i + 1])))
    return continent_list

lista = [(5,5), (6,6), (7,7)] # Testa funktionerna
string_lista = stringify(lista)
print(string_lista)
print(unstringify(string_lista))


5,5,6,6,7,7,
['5', '5', '6', '6', '7', '7', '']
[(5, 5), (6, 6), (7, 7)]


Skapa funktion som extraherar kontinenter, karta och byar och serialisera kontinent_listorna samt lägger till dem och kartan i databasen

In [10]:
def save(map, cursor):
    amount_continents = map.get_continent_amount()
    name_map = map.get_realm_name()
    (x_size, y_size) = map.get_mapsize()
    cursor.execute('''INSERT INTO maps (map_size_x, map_size_y, name, amount_continents)
                    VALUES(?,?,?,?);
                   ''', (x_size, y_size, name_map, amount_continents))
    continents = map.get_continents()
    map_id = cursor.lastrowid
    for cont in continents:
        points_list = cont.get_point_list()
        ser_points = stringify(points_list)
        cursor.execute('''INSERT INTO continents (points_serialised, id_map)
                    VALUES(?,?);
                    ''', (ser_points, map_id))
        villages = cont.get_villages().get_cities_list()
        cont_id = cursor.lastrowid
        for village in villages:
            cursor.execute('''INSERT INTO villages (x, y, id_cont)
                        VALUES(?,?,?);
                           ''', (village[0], village[1], cont_id))

Testa att spara ner en karta

In [11]:
save(ny_karta, cursor)
conn.commit() # Commita till databasen


Skapa en funktion som hämtar upp en karta och återskapar kartan

In [30]:
def load(karta_id, cursor): 
    cursor.execute('''SELECT * FROM maps
                JOIN continents ON maps.id = continents.id_map
                JOIN villages ON continents.id = villages.id_cont
                WHERE maps.id = ?
                ;
                ''', (karta_id, )) #Hämta alla kartan med karta_id
    
    data = cursor.fetchall()
    amount_continents = data[4]
    list_continents = []
    for i in range(0, amount_continents): #Hämta kontinenterna
        list_continents.append(unstringify(data[6 + i])) #Avserilisera kontinenterna
    for i in range(6 + amount_continents, len(data)): #Hämta kontinenter

    
    return Map(amount_continents, 100, 0, 0, 50, (data[1], data[2]), data[3], create_new=False, ) #Återskapa kartan
print(load(3, cursor))




[(3, 1000, 1000, 'test_karta', 5, 2, '559.6840647755765,360.2121222799718,572.6415665730138,399.6233257879918,559.2351179131064,446.48807257676344,546.9840387136327,484.18199198058096,524.0312546217618,498.63491641394285,475.18935613554606,507.6775514289336,443.83119489533334,508.4536655209947,435.66789404332906,508.98983068541656,412.73171351894786,473.54713087996765,394.30364154337326,507.64277816388653,389.47455160918105,511.3529190863834,373.04610594507295,520.1418826094057,344.7596277741035,552.0600148794204,295.95058761019214,571.4354656209867,263.975142206757,548.2567840716135,262.7327986054926,544.7040395776477,265.0005977782903,492.2929679955523,259.6265415523677,457.28043099428044,259.78820317893417,451.3918790124636,263.3331685247918,444.88580729813793,266.37373403202474,441.8944035659076,286.0236251374456,419.06994761386625,303.99267819838303,432.8831108137051,277.95312216673847,407.96621030300844,258.3896640162883,368.2346866041664,242.04426912092183,343.8923558240651,227.

Testa att spara och hämta en karta

Testa att spara och hämta flera kartor